In [1]:
%run ~/relmapping/annot/notebooks/__init__.ipynb

/mnt/home3/jj374/anaconda36/lib/python3.6/site-packages/statsmodels/compat/pandas.py:56: FutureWarning: The pandas.core.datetools module is deprecated and will be removed in a future version. Please use the pandas.tseries module instead.
  from pandas.core import datetools


os.getcwd(): /mnt/b2/scratch/ahringer/jj374/lab/relmapping


In [2]:
# Load hypersensitive sites
df_sites = pd.read_csv('annot/S1_accessible_sites/S1a_accessible_sites.tsv', sep='\t')[['chrom', 'start', 'end', 'atac_mode']]
#df_sites = df_sites.head(100)
print(len(df_sites), 'ATAC-seq sites')

42245 ATAC-seq sites


In [3]:
# Read lcap
def read_startbp_(chroms, modes, stage, strand, inner_flank=50, outer_flank=300):
    step_fwd = 'startbp_q10_reads_by_rep_fwd'
    step_rev = 'startbp_q10_reads_by_rep_rev'
    if strand == '+':
        fp_rep1 = pf('lcap_%(stage)s_rep1' % locals(), step_fwd, '.bw', 'lcap_geo')
        fp_rep2 = pf('lcap_%(stage)s_rep2' % locals(), step_fwd, '.bw', 'lcap_geo')
    else:
        fp_rep1 = pf('lcap_%(stage)s_rep1' % locals(), step_rev, '.bw', 'lcap_geo')
        fp_rep2 = pf('lcap_%(stage)s_rep2' % locals(), step_rev, '.bw', 'lcap_geo')

    (r_starts, r_ends) = ((modes - outer_flank).tolist(), (modes - inner_flank).tolist())    
    (f_starts, f_ends) = ((modes + inner_flank).tolist(), (modes + outer_flank).tolist())
    if strand == '+':
        (n_starts, n_ends) = (r_starts, r_ends)
        (p_starts, p_ends) = (f_starts, f_ends)
    else:
        (n_starts, n_ends) = (f_starts, f_ends)
        (p_starts, p_ends) = (r_starts, r_ends)

    def f_(s): return int(np.nansum(s))

    df = pd.DataFrame()
    df['rep1_ncount' % locals()] = list(map(f_, yp.mread_regions(fp_rep1, chroms, n_starts, n_ends)))
    df['rep2_ncount' % locals()] = list(map(f_, yp.mread_regions(fp_rep2, chroms, n_starts, n_ends))) 
    df['rep1_pcount' % locals()] = list(map(f_, yp.mread_regions(fp_rep1, chroms, p_starts, p_ends)))
    df['rep2_pcount' % locals()] = list(map(f_, yp.mread_regions(fp_rep2, chroms, p_starts, p_ends))) 
    return df

def read_median_(chroms, modes, stage, strand, inner_flank=50, outer_flank=300):
    step_fwd = 'coverage_q10_reads_by_rep_fwd'
    step_rev = 'coverage_q10_reads_by_rep_rev'
    if strand == '+':
        fp_rep1 = pf('lcap_%(stage)s_rep1' % locals(), step_fwd, '.bw', 'lcap_geo')
        fp_rep2 = pf('lcap_%(stage)s_rep2' % locals(), step_fwd, '.bw', 'lcap_geo')
    else:
        fp_rep1 = pf('lcap_%(stage)s_rep1' % locals(), step_rev, '.bw', 'lcap_geo')
        fp_rep2 = pf('lcap_%(stage)s_rep2' % locals(), step_rev, '.bw', 'lcap_geo')

    # median with exclusion zone
    (r_starts, r_ends) = ((modes - outer_flank).tolist(), (modes - inner_flank).tolist())    
    (f_starts, f_ends) = ((modes + inner_flank).tolist(), (modes + outer_flank).tolist())

    if strand == '+':
        (n_starts, n_ends) = (r_starts, r_ends)
        (p_starts, p_ends) = (f_starts, f_ends)
    else:
        (n_starts, n_ends) = (f_starts, f_ends)
        (p_starts, p_ends) = (r_starts, r_ends)

    def f_(a): return int(np.median(np.nan_to_num(a)))

    df = pd.DataFrame()
    df['rep1_ncount' % locals()] = list(map(f_, yp.mread_regions(fp_rep1, chroms, n_starts, n_ends)))
    df['rep2_ncount' % locals()] = list(map(f_, yp.mread_regions(fp_rep2, chroms, n_starts, n_ends))) 
    df['rep1_pcount' % locals()] = list(map(f_, yp.mread_regions(fp_rep1, chroms, p_starts, p_ends)))
    df['rep2_pcount' % locals()] = list(map(f_, yp.mread_regions(fp_rep2, chroms, p_starts, p_ends))) 
    return df

def read_merge_(df1, df2):
    def select_(s1, s2):
        fc_s1 = math.log(s1[['rep1_pcount', 'rep2_pcount']].sum() + 1, 2)\
              - math.log(s1[['rep1_ncount', 'rep2_ncount']].sum() + 1, 2)
        fc_s2 = math.log(s2[['rep1_pcount', 'rep2_pcount']].sum() + 1, 2)\
              - math.log(s2[['rep1_ncount', 'rep2_ncount']].sum() + 1, 2)
        if fc_s2 > fc_s1:
            return s2
        else:
            return s1
    return pd.DataFrame.from_records(map(lambda ir1, ir2: select_(ir1[1], ir2[1]), df1.iterrows(), df2.iterrows()))

In [4]:
def lcap_count(chroms, modes, stage):
    !date
    print(stage)
    # Naming always relative to the forward strand
    df_ = pd.DataFrame()
    df_[['lcap_%(stage)s_rep1_fwd_rcount' % locals(), 'lcap_%(stage)s_rep2_fwd_rcount' % locals(),
         'lcap_%(stage)s_rep1_fwd_fcount' % locals(), 'lcap_%(stage)s_rep2_fwd_fcount' % locals(),]] = \
            read_merge_(
                read_startbp_(chroms, modes, stage, '+'), 
                read_median_(chroms, modes, stage, '+')
            )

    df_[['lcap_%(stage)s_rep1_rev_fcount' % locals(), 'lcap_%(stage)s_rep2_rev_fcount' % locals(),
         'lcap_%(stage)s_rep1_rev_rcount' % locals(), 'lcap_%(stage)s_rep2_rev_rcount' % locals(),]] = \
            read_merge_(
                read_startbp_(chroms, modes, stage, '-'),
                read_median_(chroms, modes, stage, '-'),
            )

    return df_

df_ = pd.concat([lcap_count(df_sites['chrom'], df_sites['atac_mode'], stage)
                 for stage in itertools.islice(config['lcap_geo_by_stage'], None)], axis=1)
df_counts = pd.concat([df_sites[['chrom', 'start', 'end', 'atac_mode']], df_], axis=1)
df_counts.head()

Tue  5 Dec 11:45:01 GMT 2017
wt_emb
mread_regions True lcap_wt_emb_rep1.startbp_q10_reads_by_rep_fwd.bw.mread_regions_21bb72fb53b846566677b360465595d6.tmp
mread_regions True lcap_wt_emb_rep2.startbp_q10_reads_by_rep_fwd.bw.mread_regions_0773bbc2c0131d60951b96d636c76b10.tmp
mread_regions True lcap_wt_emb_rep1.startbp_q10_reads_by_rep_fwd.bw.mread_regions_99ed0e789f2a52f2ec107843ef1174ed.tmp
mread_regions True lcap_wt_emb_rep2.startbp_q10_reads_by_rep_fwd.bw.mread_regions_8f7834680456e618b237dfaa55578c10.tmp
mread_regions True lcap_wt_emb_rep1.coverage_q10_reads_by_rep_fwd.bw.mread_regions_e0c0e382323e3baa285a27364e700be1.tmp
mread_regions True lcap_wt_emb_rep2.coverage_q10_reads_by_rep_fwd.bw.mread_regions_fd6726a91550a786052f7258d2ce3f8d.tmp
mread_regions True lcap_wt_emb_rep1.coverage_q10_reads_by_rep_fwd.bw.mread_regions_b95c6daeddb920ea45c6c710010d9ceb.tmp
mread_regions True lcap_wt_emb_rep2.coverage_q10_reads_by_rep_fwd.bw.mread_regions_be9338fa0e721682da82f5ae1640f0eb.tmp
mread_re

mread_regions True lcap_wt_l4_rep1.coverage_q10_reads_by_rep_fwd.bw.mread_regions_32aa88db14582ebccc85c188cdd962b8.tmp
mread_regions True lcap_wt_l4_rep2.coverage_q10_reads_by_rep_fwd.bw.mread_regions_aa29e594929806427f3b032c1769d9be.tmp
mread_regions True lcap_wt_l4_rep1.coverage_q10_reads_by_rep_fwd.bw.mread_regions_af3bca58a9704f9efe01f2208bfb551a.tmp
mread_regions True lcap_wt_l4_rep2.coverage_q10_reads_by_rep_fwd.bw.mread_regions_0596480ec74a5db136042b229e992cbd.tmp
mread_regions True lcap_wt_l4_rep1.startbp_q10_reads_by_rep_rev.bw.mread_regions_9b112b5518ac6772b664850462ebd093.tmp
mread_regions True lcap_wt_l4_rep2.startbp_q10_reads_by_rep_rev.bw.mread_regions_47b9f4ae0b7e7a058268f2588379b56c.tmp
mread_regions True lcap_wt_l4_rep1.startbp_q10_reads_by_rep_rev.bw.mread_regions_7025955f908438269cc1bf370d40473b.tmp
mread_regions True lcap_wt_l4_rep2.startbp_q10_reads_by_rep_rev.bw.mread_regions_25b36521e3de4dbb4e17bb09d78c53a6.tmp
mread_regions True lcap_wt_l4_rep1.coverage_q10_read

mread_regions True lcap_glp1_d7_rep1.startbp_q10_reads_by_rep_rev.bw.mread_regions_4232092f60e40f745905a75df129bf31.tmp
mread_regions True lcap_glp1_d7_rep2.startbp_q10_reads_by_rep_rev.bw.mread_regions_08e3f0fff495a7cd02ff2c7c58f506f3.tmp
mread_regions True lcap_glp1_d7_rep1.startbp_q10_reads_by_rep_rev.bw.mread_regions_e1426077d12ef3b57b5bfffa0195a9d6.tmp
mread_regions True lcap_glp1_d7_rep2.startbp_q10_reads_by_rep_rev.bw.mread_regions_f88236837ecbc150f1d31ec3edf9ee87.tmp
mread_regions True lcap_glp1_d7_rep1.coverage_q10_reads_by_rep_rev.bw.mread_regions_7013f8d79ebf1452b7e5e9a2f6fd836a.tmp
mread_regions True lcap_glp1_d7_rep2.coverage_q10_reads_by_rep_rev.bw.mread_regions_8c84cd57312d35ce9d1dea9a4e8835d5.tmp
mread_regions True lcap_glp1_d7_rep1.coverage_q10_reads_by_rep_rev.bw.mread_regions_202b3589e40a7f0444c2790545b404a5.tmp
mread_regions True lcap_glp1_d7_rep2.coverage_q10_reads_by_rep_rev.bw.mread_regions_d7873283b9c8be6d5cba2cdf784910dd.tmp
Tue  5 Dec 12:20:09 GMT 2017
glp1_d1

,chrom,start,end,atac_mode,lcap_wt_emb_rep1_fwd_rcount,lcap_wt_emb_rep2_fwd_rcount,lcap_wt_emb_rep1_fwd_fcount,lcap_wt_emb_rep2_fwd_fcount,lcap_wt_emb_rep1_rev_fcount,lcap_wt_emb_rep2_rev_fcount,...,lcap_glp1_d10_rep1_rev_rcount,lcap_glp1_d10_rep2_rev_rcount,lcap_glp1_d14_rep1_fwd_rcount,lcap_glp1_d14_rep2_fwd_rcount,lcap_glp1_d14_rep1_fwd_fcount,lcap_glp1_d14_rep2_fwd_fcount,lcap_glp1_d14_rep1_rev_fcount,lcap_glp1_d14_rep2_rev_fcount,lcap_glp1_d14_rep1_rev_rcount,lcap_glp1_d14_rep2_rev_rcount
0,chrI,1870,2092,1981,0,0,0,1,0,0,...,38,9,1,1,1,0,8,0,12,4
1,chrI,3809,3994,3901,0,0,1,0,1,0,...,81,172,1,5,5,3,0,0,129,61
2,chrI,4302,4410,4356,1,1,0,1,2,3,...,9,3,7,8,8,7,3,1,7,6
3,chrI,11245,11465,11355,3,1,7,5,9,12,...,2,4,1,0,51,47,1,0,10,2
4,chrI,13087,13207,13147,10,11,9,11,29,26,...,10,14,101,142,114,133,19,9,9,17


In [5]:
df_counts.to_csv('annot/S2_regulatory_annotation/_lcap_jump/lcap_counts.tsv', sep='\t', index=False)

In [6]:
def read_incr(chroms, modes, stage, strand, inner_flank=50, outer_flank=300):
    step_fwd = 'startbp_q10_reads_by_rep_fwd'
    step_rev = 'startbp_q10_reads_by_rep_rev'
    #step_fwd = 'coverage_q10_reads_by_rep_fwd'
    #step_rev = 'coverage_q10_reads_by_rep_rev'
    if strand == '+':
        fp_rep1 = pf('lcap_%(stage)s_rep1' % locals(), step_fwd, '.bw', 'lcap_geo')
        fp_rep2 = pf('lcap_%(stage)s_rep2' % locals(), step_fwd, '.bw', 'lcap_geo')
    else:
        fp_rep1 = pf('lcap_%(stage)s_rep1' % locals(), step_rev, '.bw', 'lcap_geo')
        fp_rep2 = pf('lcap_%(stage)s_rep2' % locals(), step_rev, '.bw', 'lcap_geo')

    if strand == '+':
        (n_starts, n_ends) = ((modes - outer_flank).tolist(), (modes - inner_flank).tolist())
        (p_starts, p_ends) = ((modes - inner_flank).tolist(), (modes + outer_flank).tolist())
    else:
        (n_starts, n_ends) = ((modes + inner_flank).tolist(), (modes + outer_flank).tolist())
        (p_starts, p_ends) = ((modes - outer_flank).tolist(), (modes + inner_flank).tolist())

    def f_(s): return int(np.nansum(s)) # int(yp.nanmax(s))

    df = pd.DataFrame()
    df['rep1_ncount' % locals()] = list(map(f_, yp.mread_regions(fp_rep1, chroms, n_starts, n_ends)))
    df['rep2_ncount' % locals()] = list(map(f_, yp.mread_regions(fp_rep2, chroms, n_starts, n_ends))) 
    df['rep1_pcount' % locals()] = list(map(f_, yp.mread_regions(fp_rep1, chroms, p_starts, p_ends)))
    df['rep2_pcount' % locals()] = list(map(f_, yp.mread_regions(fp_rep2, chroms, p_starts, p_ends))) 
    return df

def lcap_count_incr(chroms, modes, stage):
    !date
    print(stage)
    # Naming always relative to the forward strand
    df_ = pd.DataFrame()
    df_[['lcap_%(stage)s_rep1_fwd_rcount' % locals(), 'lcap_%(stage)s_rep2_fwd_rcount' % locals(),
         'lcap_%(stage)s_rep1_fwd_fcount' % locals(), 'lcap_%(stage)s_rep2_fwd_fcount' % locals(),]] = \
                                                                    read_incr(chroms, modes, stage, '+')

    df_[['lcap_%(stage)s_rep1_rev_fcount' % locals(), 'lcap_%(stage)s_rep2_rev_fcount' % locals(),
         'lcap_%(stage)s_rep1_rev_rcount' % locals(), 'lcap_%(stage)s_rep2_rev_rcount' % locals(),]] = \
                                                                    read_incr(chroms, modes, stage, '-')
    return df_

df_ = pd.concat([lcap_count_incr(df_sites['chrom'], df_sites['atac_mode'], stage)
                 for stage in itertools.islice(config['lcap_geo_by_stage'], None)], axis=1)
df_counts = pd.concat([df_sites[['chrom', 'start', 'end', 'atac_mode']], df_], axis=1)

Tue  5 Dec 12:28:08 GMT 2017
wt_emb
mread_regions True lcap_wt_emb_rep1.startbp_q10_reads_by_rep_fwd.bw.mread_regions_21bb72fb53b846566677b360465595d6.tmp
mread_regions True lcap_wt_emb_rep2.startbp_q10_reads_by_rep_fwd.bw.mread_regions_0773bbc2c0131d60951b96d636c76b10.tmp
mread_regions True lcap_wt_emb_rep1.startbp_q10_reads_by_rep_fwd.bw.mread_regions_7e6b835aa05e5099de40e32424d51406.tmp
mread_regions True lcap_wt_emb_rep2.startbp_q10_reads_by_rep_fwd.bw.mread_regions_5a6a542318a887c2ebf298234d1040b8.tmp
mread_regions True lcap_wt_emb_rep1.startbp_q10_reads_by_rep_rev.bw.mread_regions_09750fdabf54ef36afa8efbfa8e76a25.tmp
mread_regions True lcap_wt_emb_rep2.startbp_q10_reads_by_rep_rev.bw.mread_regions_638c8f2206ed259a001fc3edb3e6ad06.tmp
mread_regions True lcap_wt_emb_rep1.startbp_q10_reads_by_rep_rev.bw.mread_regions_f7c3265898b585c9beb09192f8ce7609.tmp
mread_regions True lcap_wt_emb_rep2.startbp_q10_reads_by_rep_rev.bw.mread_regions_248ef6b0b489a55bbe3d302556d61e5f.tmp
Tue  5 Dec 1

mread_regions True lcap_glp1_d7_rep2.startbp_q10_reads_by_rep_fwd.bw.mread_regions_92ebee152fd06df14af9d7fc4c63f7b9.tmp
mread_regions True lcap_glp1_d7_rep1.startbp_q10_reads_by_rep_rev.bw.mread_regions_4232092f60e40f745905a75df129bf31.tmp
mread_regions True lcap_glp1_d7_rep2.startbp_q10_reads_by_rep_rev.bw.mread_regions_08e3f0fff495a7cd02ff2c7c58f506f3.tmp
mread_regions True lcap_glp1_d7_rep1.startbp_q10_reads_by_rep_rev.bw.mread_regions_5ce82f2674cd0687f4703c2f48792cc5.tmp
mread_regions True lcap_glp1_d7_rep2.startbp_q10_reads_by_rep_rev.bw.mread_regions_fc73dda07df9cce0318400645fe09277.tmp
Tue  5 Dec 12:29:22 GMT 2017
glp1_d10
mread_regions True lcap_glp1_d10_rep1.startbp_q10_reads_by_rep_fwd.bw.mread_regions_447a16ec176c09391cc39c87c99cf451.tmp
mread_regions True lcap_glp1_d10_rep2.startbp_q10_reads_by_rep_fwd.bw.mread_regions_d2206ce1b75f4d95494d7edb64561f8b.tmp
mread_regions True lcap_glp1_d10_rep1.startbp_q10_reads_by_rep_fwd.bw.mread_regions_d3e9df7d5130354a0433aa7dccddf240.tmp

In [7]:
df_counts.to_csv('annot/S2_regulatory_annotation/_lcap_jump/lcap_counts_incr.tsv', sep='\t', index=False)